In [ ]:
!pip install accelerate>=0.20.3 transformers
!pip install torch

In [ ]:
import accelerate
import torch
from transformers import AutoTokenizer, FuyuForCausalLM, FuyuImageProcessor, FuyuProcessor
from PIL import Image
import requests

In [ ]:
device = "cpu"

dtype = torch.bfloat16 if device != "mps" else torch. float16

model_id = "adept/fuyu-8b"
tokenizer = AutoTokenizer.from_pretrained(model_id)
Fuyu_Model = FuyuForCausalLM.from_pretrained(model_id, device_map=device, torch_dtype=dtype)
Fuyo_processor = FuyuProcessor(image_processor=FuyuImageProcessor(), tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/848 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/6.13M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.5M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/58.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.93G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/8.88G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

In [ ]:
prompt = "Generate a coco-style caption.\n"

In [ ]:
image_link = 'https://www.thoughtco.com/thmb/O_G0KJHjLi6nG96Q-vGrs20J8go=/1500x0/filters:no_upscale():max_bytes(150000):strip_icc()/GettyImages-128278067-8a45e1211a4841ae83e0c23245664837.jpg'
response = requests.get(image_link)
image = Image.open(requests.get(image_link, stream=True).raw)

In [ ]:
import pickle
with open('/content/paragraphs.obj', 'rb') as pickleFile:
    text = pickle.load(pickleFile)

text[0]

{'paragraph': ' Editors’ Introduction:  THIS petition was presented by 1,500 persons on Dec. 11, 1640, on behalf of 15,000 Londoners who had signed it. The Commons postponed its consideration, but in the following February referred it to a committee. The petition must be distinguished from the Root and Branch Bill said to have been drawn up by St. John, and presented to Parliament by Vane and Cromwell in May, 1641. The bill was dropped in the House of Commons, and finally abandoned after long debates in August. ',
 'nr': 1,
 'bookID': 'train_files\\15-000-londoners-the-root-and-branch-petition.epub'}

In [ ]:
inputs = Fuyo_processor(text=prompt, images=image, return_tensors="pt")#.to('device')

# autoregressively generate text
generation_output = Fuyu_Model.generate(**inputs, max_new_tokens=7)
generation_text = Fuyo_processor.batch_decode(generation_output[:, -7:], skip_special_tokens=True)

Setting `pad_token_id` to `eos_token_id`:71013 for open-end generation.


KeyboardInterrupt: 

In [ ]:
generation_text

['A large group of people are protesting outside.']

In [ ]:
def get_text(image_path, prompt):
    image = Image.open(image_path).convert('RGB')
    inputs = Fuyo_processor(text=prompt, images=image, return_tensors="pt").to(device)
    generation_output = Fuyu_Model.generate(**inputs, max_new_tokens = 7)
    generation_text = Fuyo_processor.batch_decode(generation_output[:, -7:], skip_special_token=True)
    return generation_text

In [ ]:
def search_tfidf_image(image_path, prompt, k=1):
    im_text = get_text(image_path, prompt)[0]
    print(im_text)
    return search_tfidf(im_text, k)

In [ ]:
search_tfidf_image('/content/ChildrenGames.jpg', prompt, 10)

RuntimeError: Expected one of cpu, cuda, ipu, xpu, mkldnn, opengl, opencl, ideep, hip, ve, fpga, ort, xla, lazy, vulkan, mps, meta, hpu, mtia, privateuseone device type at start of device string: auto